In [1]:
%env DB_HOST=mongodb://localhost/openpath_prod_ccebikes
%env DB_RESULT_LIMIT=1000000
import emission.core.get_database as edb
#import emission.storage.timeseries.aggregate_timeseries as esta
import emission.storage.timeseries.builtin_timeseries as estb
import emission.core.get_database as gdb

# Import necessary libraries and modules
import emission.core.get_database as edb
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.timeseries.builtin_timeseries as estb
import emission.storage.decorations.analysis_timeseries_queries as esda
import emission.core.wrapper.entry as ecwe
import emission.storage.decorations.trip_queries as esdt
import emission.storage.timeseries.timequery as estt
import pandas as pd
from datetime import datetime, timedelta
import pytz
import pprint
import os

env: DB_HOST=mongodb://localhost/openpath_prod_ccebikes
env: DB_RESULT_LIMIT=1000000
Config file not found, returning a copy of the environment variables instead...
Retrieved config: {'DB_HOST': 'mongodb://localhost/openpath_prod_ccebikes', 'DB_RESULT_LIMIT': '1000000'}
Connecting to database URL mongodb://localhost/openpath_prod_ccebikes


In [7]:
ts = esta.TimeSeries.get_aggregate_time_series()
#df = ts.get_data_df(
#                "stats/pipeline_time")

df2 = list(ts.find_entries(["stats/pipeline_time"], extra_query_list=[{"data.name": "STORE_USER_STATS"}]))

df=pd.DataFrame(df2)
# Flatten 'metadata' and 'data' fields
if 'metadata' in df.columns:
    metadata_expanded = pd.json_normalize(df['metadata'])
    df = pd.concat([df.drop(columns=['metadata']), metadata_expanded], axis=1)

if 'data' in df.columns:
    data_expanded = pd.json_normalize(df['data'])
    df = pd.concat([df.drop(columns=['data']), data_expanded], axis=1)

#print(df)

if 'write_ts' in df.columns:
    df['write_ts_dt'] = pd.to_datetime(df['write_ts'], unit='s')

# Find the earliest and latest entries
earliest_entry = df.loc[df['write_ts'].idxmin()]
latest_entry = df.loc[df['write_ts'].idxmax()]

# Display the results
print("Earliest Entry:")
print(earliest_entry)

print("\nLatest Entry:")
print(latest_entry)

2024-12-09 11:18:06,478 - root - DEBUG - curr_query = {'invalid': {'$exists': False}, '$or': [{'metadata.key': 'stats/pipeline_time'}], 'data.name': 'STORE_USER_STATS'}, sort_key = None
2024-12-09 11:18:06,480 - root - DEBUG - orig_ts_db_keys = ['stats/pipeline_time'], analysis_ts_db_keys = []
2024-12-09 11:18:08,350 - root - DEBUG - finished querying values for ['stats/pipeline_time'], count = 1064
2024-12-09 11:18:08,351 - root - DEBUG - finished querying values for [], count = 0
2024-12-09 11:18:08,353 - root - DEBUG - orig_ts_db_matches = 1064, analysis_ts_db_matches = 0


Earliest Entry:
_id                                    672b9a210783e03ac9b30f7c
user_id                    6f836860-8b7b-4e1e-940f-7c6eb2e793e3
key                                         stats/pipeline_time
platform                                                 server
write_ts                                      1730910753.231596
time_zone                                   America/Los_Angeles
write_fmt_time                 2024-11-06T08:32:33.231596-08:00
write_local_dt.year                                        2024
write_local_dt.month                                         11
write_local_dt.day                                            6
write_local_dt.hour                                           8
write_local_dt.minute                                        32
write_local_dt.second                                        33
write_local_dt.weekday                                        2
write_local_dt.timezone                     America/Los_Angeles
name                    